In [1]:
import copy
import numpy as np
import pandas as pd
import pathlib

from typing import DefaultDict, List, Tuple, Set
from collections import defaultdict

In [2]:
# All CONSTANTS used in the notebook

# Set the path to the Collusion Clustering dataset
PATH = pathlib.Path('./dataset.csv')
MAX_DATASET_SIZE = 10
DELTA = 0.000001 # Stability

# Alias
Graph = DefaultDict[int, List[Tuple[int, int]]]

In [3]:
def process_dataset(path: pathlib.Path):
    """Read and process dataset"""
    
    df = pd.read_csv(path)
    df.columns = ['sid', 'bid', 'amount']
    return df

In [4]:
# Setting seed
np.random.seed(42)
df = process_dataset(PATH)

In [5]:
graph = defaultdict(list)
for index, row in df.iterrows():
    sid, bid, amount = row
    graph[sid].append((bid, amount))

In [6]:
class CollusionClustering:
    def __init__(self, k: int, *, m: int, h: int):
        assert 1 <= m <= k
        assert 0 <= h <= 100
        self.k = k
        self.m = m
        self.h = h
    
    def _get_sfg(self, graph: Graph):
        """Return a Stock Flow Graph with max k neighbours"""
        sfg = copy.copy(graph)
        for v in sfg.keys():
            sfg[v].sort(key=lambda x: x[1], reverse=True)

            # k-nearest neighbours of v
            sfg[v] = sfg[sid][:self.k]
        return sfg

    def _get_internal_trade_val(self, c):
        total = 0 
        for u in c:
            for v, amount in self._G[u]:
                if u != v and v in c:
                    total += amount
        return total 

    def _get_collusion_level(self, c1: Set[int], c2: Set[int]):
        assert c1.isdisjoint(c2)

        c = c1 | c2

        internal_trade = self._get_internal_trade_val(c)
        external_trade = self._get_external_trade_val(c) + DELTA
                    
        return internal_trade / external_trade

    def _get_cluster_pairs(self, S):
        shape = (len(S), len(S))
        return sorted([(x, y, self._get_collusion_level(S[x], S[y])) for x, y in np.ndindex(shape) if x != y], key=lambda t: t[2], reverse=True)

    def _km_compatibility(self, p, D):
        knn_p = self._G[p]
        return len(knn_p) >= min(self.m, len(D))

    def _khm_compatibility(self, C, D):
        c_compatible = sum([self._km_compatibility(c, D) for c in C])
        d_compatible = sum([self._km_compatibility(d, C) for d in D])
        if c_compatible >= int(self.h * len(C) / 100) and d_compatible >= int(self.h * len(D) / 100):
            return True
        return False

    def fit(self, graph: Graph):
        # S contains collusion sets
        self._G = self._get_sfg(graph)
        
        # Initially every vertex is a singleton cluster
        S = list(map(lambda x: {x}, self._G.keys()))

        while True:
            B = self._get_cluster_pairs(S)
            found = False
            for C_i, D_i, Lc in B:
                if Lc > 0 and self._khm_compatibility(S[C_i], S[D_i]):
                    S[C_i].update(S[D_i])
                    S[D_i], S[-1] = S[-1], S[D_i]
                    S.pop()
                    found = True
                    print('Merging')
                    break
            if not found:
                break
        return S

    def _get_external_trade_val(self, c):
        total = 0
        for u in self._G.keys():
            for v, amount in self._G[u]:
                if u != v and ((v in c) != (u in c)):
                    total += amount
        return total 


In [7]:
S = CollusionClustering(k=4, m=1, h=50).fit(graph)

Merging
Merging
Merging
Merging
Merging
Merging
Merging


In [8]:
display(S)

[{6729, 13548, 43798, 50762, 51632, 86038, 87317, 96317}]